In [ ]:
# adding kernel
# https://medium.com/@lerouxpierre/how-to-add-kernels-to-your-jupyter-notebooks-within-azure-ml-studio-6495895c9d4a
# fix "kernel reset" error
# https://stackoverflow.com/questions/77902093/error-kernel-reset-in-azure-ml-compute-jupyterlab

# %conda config --append channels anaconda
# %conda config --add channels conda-forge
# %conda config --add channels conda

# to allow conda managing pypi packages
%conda config --set pip_interop_enabled True

# %conda update -n base conda
# %conda install python=3.12 dask=2024.12.0 adlfs=2024.7.0 pytorch=2.5.1 scikit-learn=1.5.1 pyarrow=18.0.0 pandas=2.2.3 numpy=2.2.0

%conda install scikit-learn=1.5.1 onnx=1.17.0
%pip install onnxscript


In [ ]:
%conda config --show

In [ ]:
%conda list
# %conda --version

In [17]:
# import dask.dataframe as dd
import pandas as pd


In [18]:
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader


In [ ]:
storage_options = {
    "account_name": "lnglattzw21586036186",
    "account_key" : "REMOVE",
}

df = pd.read_parquet(
    [
        "data2/lng-lat-tz/data0_0.parquet",
        # "data2/lng-lat-tz/data1_0.parquet",
    ],
    # "az://lng-lat-tz/data1*.parquet",
    # aggregate_files     = True,
    # split_row_groups    = 'adaptive',
    # storage_options     = storage_options,
    # dtype={'lng': 'float', 'lat': 'float', 'tz': 'string'}
    # use_nullable_dtypes = True,
)

# test
df.head()

In [20]:
# test azure with dask or pandas
# storage_options = {'account_name': 'azureopendatastorage'}
# ddf = dd.read_parquet('az://nyctlc/green/puYear=2019/puMonth=*/*.parquet', storage_options=storage_options)
# ddf.head()

In [21]:
import torch
import torch.nn as nn

class NeuralNetwork(nn.Module):
    def __init__(self, out_neurons):
        super().__init__()
        power_l1 = 9
        power_l2 = 9
        power_l3 = 9
        power_l4 = 9
        power_l5 = 9
        power_l6 = 9
        power_l7 = 9
        power_l8 = 9
        power_l9 = 9
        power_l10 = 9
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(2, 2 ** power_l1),
            nn.ReLU(),
            nn.Linear(2 ** power_l1, 2 ** power_l2),
            nn.ReLU(),
            nn.Linear(2 ** power_l2, 2 ** power_l3),
            nn.ReLU(),
            nn.Linear(2 ** power_l3, 2 ** power_l4),
            nn.ReLU(),
            nn.Linear(2 ** power_l4, 2 ** power_l5),
            nn.ReLU(),
            # nn.Linear(2 ** power_l5, 2 ** power_l6),
            # nn.ReLU(),
            # nn.Linear(2 ** power_l6, 2 ** power_l7),
            # nn.ReLU(),
            # nn.Linear(2 ** power_l7, 2 ** power_l8),
            # nn.ReLU(),
            # nn.Linear(2 ** power_l8, 2 ** power_l9),
            # nn.ReLU(),
            # nn.Linear(2 ** power_l9, 2 ** power_l10),
            # nn.ReLU(),
            nn.Linear(2 ** power_l10, out_neurons),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits


In [22]:
def train_loop(batch_size, dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    # Set the model to training mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * batch_size + len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")


In [23]:
# memorize 
# df = df.compute()

In [ ]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)

# validate device
device

In [25]:
y = df.drop(['lng', 'lat'], axis=1)
X = df.drop(['tz'], axis=1)

# scaler = MinMaxScaler()
# X_normalized = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

l_t = LabelEncoder()
y_label_encoding = y.apply(lambda x : l_t.fit_transform(x))

X_train, X_test, y_train, y_test = train_test_split(X, y_label_encoding, test_size = 0.2, shuffle= True)

X_train = torch.FloatTensor(X_train.values).to(device)
X_test = torch.FloatTensor(X_test.values).to(device)
y_train = torch.flatten(torch.LongTensor(y_train.values)).to(device)
y_test = torch.flatten(torch.LongTensor(y_test.values)).to(device)

training_data = TensorDataset(X_train, y_train)
test_data = TensorDataset(X_test, y_test)


In [ ]:
learning_rate = 1e-3 # 4
batch_size = 32 * 8 # 32
epochs = 1 # 100
class_count = df['tz'].value_counts().count()
model = NeuralNetwork(class_count).to(device)

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(batch_size, train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)

dummy_input = torch.randn(1, 2)

torch.onnx.export(
    model.to('cpu'),                  # model to export
    dummy_input,        # inputs of the model,
    "model_l5_d0_e1.onnx",        # filename of the ONNX model
    input_names=["lng-lat"],  # Rename inputs for the ONNX model
    dynamo=True             # True or False to select the exporter to use
)

In [27]:
# adding ArgMax & Label Encoder after
# https://stackoverflow.com/questions/72348419/how-to-add-argmax-to-the-end-of-onnx-model

l2-2-r-data1-cpu(ef-box), batch=64, lr=0.001/Adam, epochs=1:
- loading: ?
- Accuracy: 32.7%, Avg loss: 2.712187
- runtime: ?

l2-2-r-data1-gpu, batch=64, lr=0.001/Adam, epochs=1:
- loading: ?
- Accuracy: 31.7%, Avg loss: 2.649778
- runtime: 2:41

l2-2-r-data2-gpu, batch=64, lr=0.001/Adam, epochs=1:
- loading: 2:50
- Accuracy: none
- runtime: canceled after 1hr, est 4hrs 
- cpu: 100%
- ram: 32%

l2-2-r-data1-gpu, batch=32x8, lr=0.001/Adam, epochs=1:
- loading: 1sec
- Accuracy: 30.7%, Avg loss: 2.920631
- runtime: 1:18
- cpu: 100%
- ram: 15%

l2-1K-r-data1-gpu, batch=32x8, lr=0.001/Adam, epochs=1:
- loading: NA
- Accuracy: 79.1%, Avg loss: 0.598570
- runtime: 1:29
- cpu: 100%
- ram: 15%

l2-1M-r-data1-gpu, batch=32x8, lr=0.001/Adam, epochs=1:
- loading: 1sec
- loss: 10
- runtime: canceled after 8min
- cpu: 100%
- ram: 1%

l2-128-r-data1-gpu, batch=32x8, lr=0.001/Adam, epochs=1:
- loading: 1sec
- Accuracy: 69.8%, Avg loss: 0.894220
- runtime: 1:27
- cpu: 100%
- ram: 1%

l2-4K-r-data1-gpu, batch=32x8, lr=0.001/Adam, epochs=1:
- loading: 1sec
- Accuracy: 78.8%, Avg loss: 0.582083
- runtime: 1:29
- cpu: 100%
- ram: 1%

l2-(512-r)x2-data1-gpu, batch=32x8, lr=0.001/Adam, epochs=1:
- loading: 1sec
- Accuracy: 82.4%, Avg loss: 0.473214
- runtime: 1:29
- cpu: 100%
- ram: 1%

l2-(512-r)x2-data1-gpu, batch=32x8, lr=0.001/Adam, epochs=10:
- loading: 1sec
- Accuracy: 89.0%, Avg loss: 0.293857
- runtime: 14:38
- cpu: 100%
- ram: 1%

l2-(512-r)x2-data1-gpu, batch=32x8, lr=0.1/Adam, epochs=1:
- loading: 1sec
- Accuracy: 3.6%, Avg loss: 4.470265
- runtime: 1:29
- cpu: 100%
- ram: 1%

l2-(512-r)x2-data1-gpu, batch=32x8, lr=0.0001/Adam, epochs=1:
- loading: 1sec
- Accuracy: 74.3%, Avg loss: 0.783508
- runtime: 1:29
- cpu: 100%
- ram: 1%

l2-(512-r)x2-data1-gpu, batch=32, lr=0.001/Adam, epochs=1:
- loading: 1sec
- Accuracy: 70.3%, Avg loss: 0.942639
- runtime: 5:02
- cpu: 100%
- ram: 1%

l2-(512-r)x2-data1-gpu, batch=32x16, lr=0.001/Adam, epochs=1:
- loading: 1sec
- Accuracy: 79.8%, Avg loss: 0.530371
- runtime: 1:05
- cpu: 100%
- ram: 1%

l2-(512-r)x2-data1-gpu, batch=32x4, lr=0.001/Adam, epochs=1:
- loading: 1sec
- Accuracy: 78.7%, Avg loss: 0.557121
- runtime: 1:50
- cpu: 100%
- ram: 1%

l2-(512-r)x3-data1-gpu, batch=32x8, lr=0.001/Adam, epochs=1:
- loading: 1sec
- Accuracy: 78.1%, Avg loss: 0.532118
- runtime: 1:30
- cpu: 100%
- ram: 1%

l2-(512-r)x2-data1-gpu, batch=32x8, lr=0.001/Adam, epochs=1 (reproduce prev result):
- loading: 1sec
- Accuracy: 81.9%, Avg loss: 0.494453
- runtime: 1:26
- cpu: 100%
- ram: 1%

l2-(512-r)x5-data1-gpu, batch=32x8, lr=0.001/Adam, epochs=1:
- loading: 1sec
- Accuracy: 84.7%, Avg loss: 0.388047
- runtime: 1:42
- cpu: 100%
- ram: 1%

l2-(512-r)x10-data1-gpu, batch=32x8, lr=0.001/Adam, epochs=1:
- loading: 1sec
- Accuracy: 80.7%, Avg loss: 0.532438
- runtime: 2:04
- cpu: 100%
- ram: 1%

l2-(512-r)x5-data2-gpu, batch=32x8, lr=0.001/Adam, epochs=1:
- loading: 2:59
- Accuracy: 95.2%, Avg loss: 0.118907
- runtime: 2:48:00
- cpu: 100%
- ram: 32%

l2-(512-r)x10-data0+1-gpu, batch=32x8, lr=0.001/Adam, epochs=1:
- loading: 1sec
- Accuracy: 81.3%, Avg loss: 0.457877
- runtime: 2:08
- cpu: 100%
- ram: 1%

l2-(512-r)x10-data0+1-gpu, batch=32x8, lr=0.001/Adam, epochs=1, normalized:
- loading: 1sec
- Accuracy: 78.9%, Avg loss: 0.629528
- runtime: 2:06
- cpu: 100%
- ram: 1%

l2-(4K-r)x10-data0+1-gpu, batch=32x8, lr=0.001/Adam, epochs=5, normalized:
- loading: 1sec
- Accuracy: 88.6%, Avg loss: 0.299482
- runtime: 58:00
- cpu: 100%
- ram: 2%

l2-(512-r)x7-data0+1-gpu, batch=32x8, lr=0.001/Adam, epochs=1:
- loading: 1sec
- Accuracy: 84.5%, Avg loss: 0.385082
- runtime: 1:43
- cpu: 100%
- ram: 1%

l2-(512-r)x6-data0+1-gpu, batch=32x8, lr=0.001/Adam, epochs=1:
- loading: 1sec
- Accuracy: 81.3%, Avg loss: 0.474233
- runtime: 1:38
- cpu: 100%
- ram: 2%

l2-(512-r)x4-data0+1-gpu, batch=32x8, lr=0.001/Adam, epochs=1:
- loading: 1sec
- Accuracy: 77.6%, Avg loss: 0.563156
- runtime: 1:29
- cpu: 100%
- ram: 1%
